In [11]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers.convolutional import Convolution2D
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.pooling import MaxPooling2D
from keras.layers.pooling import AveragePooling2D
from keras.layers import Cropping2D
from keras.layers import Lambda
from keras.callbacks import ModelCheckpoint
from keras.utils import Sequence
from tensorflow.python.client import device_lib
from scipy.misc import imread
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.utils.vis_utils import plot_model as plot
from math import e, sqrt, pi

import random
import numpy as np
import csv
import os
import tensorflow as tf
import re


# def has_gpu():
#     local_device_protos = device_lib.list_local_devices()
#     return True if [x.name for x in local_device_protos if x.device_type == 'GPU'] != [] else False

# # If has gpu, control GPU Memory
# if has_gpu():
#     gpu_options = tf.GPUOptions(allow_growth=True)
#     sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
#     tf.keras.backend.set_session(sess)

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


SystemError: ignored

In [0]:
!ls

sample_data


In [12]:
!pwd
!cp -r drive/Develop/formula_trend/data/ .

/content


In [13]:
folder = os.getcwd()
img_path = folder + '/data/img/'
driving_log = folder + '/data/driving_log.csv'
# img_path = './data/img/'
# driving_log = './data/driving_log.csv'

# folder = 'drive/Develop/formula_trend/'
# img_path = 'drive/Develop/formula_trend/data/img/'
# driving_log = 'drive/Develop/formula_trend/data/driving_log.csv'


def get_img_file_name(path):
    return os.path.split(path)[-1]

def gauss(x, mu=0, sigma=0.18):
    """
    utility function to calculate gaussion function
    """
    a = 1/(sigma*sqrt(2*pi))
    return a*e**(-0.5*(float(x-mu)/sigma)**2)

random_state = 42
current_dir = os.getcwd()
max_gauss = gauss(0)

images = []
steerings = []


def _should_drop(steering, drop_rate=0.7):
    """
    Randomly drop some data that drives around 0 degree
    (in a normal distribution manager.)

    for more detail please see writeup_report.md
    """
    steer_drop_rate = drop_rate * gauss(steering) / max_gauss
    return random.random() < steer_drop_rate

# Preprocess:
# read driving_log.csv and prepare training dataset
print('Reading data from %s ...' % driving_log)

with open(driving_log, 'r') as f:
    # there is no header name
    for row in f.readlines():
        (center, steering, throttle,
            brake, speed, time, lap) = row.split(",")
        steering = float(steering)
        center = get_img_file_name(center)
        print(center)
        # randomly skip some data driving strait
        if _should_drop(steering):
            continue
        else:
            images.append(center)
            steerings.append(steering)


Reading data from /content/data/driving_log.csv ...
center_2018_08_29_11_39_34_386.jpg
center_2018_08_29_11_39_34_452.jpg
center_2018_08_29_11_39_34_519.jpg
center_2018_08_29_11_39_34_585.jpg
center_2018_08_29_11_39_34_652.jpg
center_2018_08_29_11_39_34_719.jpg
center_2018_08_29_11_39_34_785.jpg
center_2018_08_29_11_39_34_852.jpg
center_2018_08_29_11_39_34_919.jpg
center_2018_08_29_11_39_34_987.jpg
center_2018_08_29_11_39_35_052.jpg
center_2018_08_29_11_39_35_119.jpg
center_2018_08_29_11_39_35_186.jpg
center_2018_08_29_11_39_35_252.jpg
center_2018_08_29_11_39_35_319.jpg
center_2018_08_29_11_39_35_385.jpg
center_2018_08_29_11_39_35_452.jpg
center_2018_08_29_11_39_35_519.jpg
center_2018_08_29_11_39_35_585.jpg
center_2018_08_29_11_39_35_652.jpg
center_2018_08_29_11_39_35_719.jpg
center_2018_08_29_11_39_35_785.jpg
center_2018_08_29_11_39_35_852.jpg
center_2018_08_29_11_39_35_919.jpg
center_2018_08_29_11_39_35_985.jpg
center_2018_08_29_11_39_36_053.jpg
center_2018_08_29_11_39_36_118.jpg
cen

In [0]:
def prepare_data(img_name, steering, random_flip=False, img_path=img_path):
    """Load image data (and randomly flip if required)"""
    img = imread(img_path+img_name).astype(np.float32)

    if random_flip and random.random() > 0.5:
        img = np.fliplr(img)
        steering = -steering

    return img, steering


# generator function for training and validating
def batches(img_names, steerings, batch_size=128, training=False):
    """Generator that generates data batch by batch
    validating: indicates generator is in training mode
    """
    # check input data integrity
    num_imgs = img_names.shape[0]
    num_steerings = steerings.shape[0]
    assert num_imgs == num_steerings

    while True:
        for offset in range(0, num_imgs, batch_size):
            X_batch = []
            y_batch = []

            stop = offset + batch_size
            img_names_b = img_names[offset:stop]
            steerings_b = steerings[offset:stop]

            for i in range(img_names_b.shape[0]):
                img, steering = prepare_data(
                    img_names_b[i], steerings_b[i], random_flip=training)
                X_batch.append(img)
                y_batch.append(steering)

            X_batch = np.array(X_batch)
            y_batch = np.array(y_batch)
            yield X_batch, y_batch


def _normalize(X):
    a = -0.1
    b = 0.1
    x_min = 0
    x_max = 255
    return a + (X - x_min) * (b - a) / (x_max - x_min)


def model_builder():
  
    model_save_name = folder + '/model.h5'
    if os.path.exists(model_save_name):
        return load_model(model_save_name)
  
    """
    Define and compile model
    """
    model = Sequential()
    # crop image 3@160x320 -> 3@80x320
    model.add(Cropping2D(
        cropping=((50, 30), (0, 0)),
        input_shape=(240, 320, 3)))
    # normalize rgb data [0~255] to [-1~1]
    model.add(Lambda(_normalize))

    # Convolution layers
    # Let network learn it's own color spaces
    model.add(Convolution2D(3, (1, 1)))
    # reshape image by 1/4 using average pooling later
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
    # 3@40x160
    model.add(Convolution2D(24, (5, 5), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    # 24@18x78
    model.add(Convolution2D(36, (5, 5), activation='relu'))
    model.add(MaxPooling2D((2, 2), (1, 2)))
    # 36@7x37
    model.add(Convolution2D(48, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    # 48@5x35
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    # 64@3x33
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    # 64@1x31

    # Fully connected layers
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))

    model.compile('Adam', 'mse', metrics=['mse'])
    return model 



In [15]:
print('Shuffling and Train test split ...')
images, steerings = shuffle(images, steerings, random_state=random_state)
paths_train, paths_test, steerings_train, steerings_test = train_test_split(
    images, steerings, test_size=0.2, random_state=random_state)


Shuffling and Train test split ...


In [16]:
# check testing data ok
paths_test = np.array(paths_test)
steerings_test = np.array(steerings_test)
assert paths_test.shape[0] == steerings_test.shape[0]
print('validation set size %d' % steerings_test.shape[0])

# check training data ok
paths_train = np.array(paths_train)
steerings_train = np.array(steerings_train)
assert paths_train.shape[0] == steerings_train.shape[0]
print('training set size %d' % paths_train.shape[0])

validation set size 59
training set size 233


In [17]:
print('Creating model...')

model = model_builder()
#plot(model, to_file='model.png', show_shapes=True, show_layer_names=False)

# Train model
print('Validating traing / testing data size ...')
assert paths_train.shape[0] == steerings_train.shape[0]
assert paths_test.shape[0] == steerings_test.shape[0]
print('Data looks good!')

train_size = paths_train.shape[0]
test_size = paths_test.shape[0]
batch_size = 32
init_epoch = 1000
nb_epochs = 1050

class SequenceData(Sequence):
    def __init__(self, x_set, y_set, batch_size=128, training=False):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.training = training

        # check input data integrity
        assert self.x.shape[0] == self.y.shape[0]

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        X_batch = []
        y_batch = []

        start = idx * self.batch_size
        end = (idx + 1) * self.batch_size

        img_names_b = self.x[start:end]
        steerings_b = self.y[start:end]

        for i in range(img_names_b.shape[0]):
            img, steering = prepare_data(
                img_names_b[i], steerings_b[i], random_flip=self.training)
            X_batch.append(img)
            y_batch.append(steering)

        return np.array(X_batch), np.array(y_batch)

print('Start training... batch size %d' % batch_size)
# train_generator = batches(
#     paths_train, steerings_train, batch_size=batch_size, training=True)
# test_generator = batches(paths_test, steerings_test, batch_size=batch_size)
train_step = int(np.ceil(train_size / float(batch_size)))
validate_step = int(np.ceil(test_size / float(batch_size)))
train_generator = SequenceData(paths_train, steerings_train, batch_size=batch_size, training=True)
test_generator = SequenceData(paths_test, steerings_test, batch_size=batch_size)

save_checkpoint = ModelCheckpoint('checkpoint.{epoch:02d}.h5', period=100)

print("Model fitting...")
model.fit_generator(
    train_generator, epochs=nb_epochs, initial_epoch=init_epoch,
    validation_data=test_generator,
    callbacks=[save_checkpoint])
print('Finished!')


Creating model...
Validating traing / testing data size ...
Data looks good!
Start training... batch size 32
Model fitting...
Epoch 1001/1050


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


8/8 [==============================] - 16s 2s/step - loss: 125.2724 - mean_squared_error: 125.2724 - val_loss: 64.1071 - val_mean_squared_error: 64.1071
Epoch 1002/1050
8/8 [==============================] - 15s 2s/step - loss: 123.6367 - mean_squared_error: 123.6367 - val_loss: 63.9435 - val_mean_squared_error: 63.9435
Epoch 1003/1050
8/8 [==============================] - 15s 2s/step - loss: 120.8216 - mean_squared_error: 120.8216 - val_loss: 39.5895 - val_mean_squared_error: 39.5895
Epoch 1004/1050
8/8 [==============================] - 15s 2s/step - loss: 110.8200 - mean_squared_error: 110.8200 - val_loss: 39.7926 - val_mean_squared_error: 39.7926
Epoch 1005/1050
8/8 [==============================] - 15s 2s/step - loss: 114.1872 - mean_squared_error: 114.1872 - val_loss: 47.8489 - val_mean_squared_error: 47.8489
Epoch 1006/1050
8/8 [==============================] - 15s 2s/step - loss: 110.3893 - mean_squared_error: 110.3893 - val_loss: 32.8825 - val_mean_squared_error: 32.8825
Ep

In [18]:
# Save trained model
model_save_name = folder + '/model.h5'
print('Saving model...')
model.save(model_save_name)
print('Model has been save as %s' % model_save_name)

Saving model...
Model has been save as /content/model.h5


In [19]:
!ls -al

total 12120
drwxr-xr-x 1 root root     4096 Aug 29 04:46 .
drwxr-xr-x 1 root root     4096 Aug 29 02:07 ..
-rw-r--r-- 1 root root     2495 Aug 29 02:21 adc.json
drwxr-xr-x 1 root root     4096 Aug 29 02:21 .config
drwxr-xr-x 3 root root     4096 Aug 29 03:52 data
drwxr-xr-x 2 root root     4096 Aug 29 02:21 drive
-rw-r--r-- 1 root root 12373792 Aug 29 04:46 model.h5
drwxr-xr-x 2 root root     4096 Aug 22 16:42 sample_data


In [0]:
!cp *.h5 drive/Develop/formula_trend/models

In [0]:
!pip list

Package                  Version  
------------------------ ---------
absl-py                  0.4.0    
altair                   2.2.2    
astor                    0.7.1    
beautifulsoup4           4.6.3    
bleach                   2.1.4    
cachetools               2.1.0    
certifi                  2018.8.24
chardet                  3.0.4    
crcmod                   1.7      
cycler                   0.10.0   
decorator                4.3.0    
entrypoints              0.2.3    
future                   0.16.0   
gast                     0.2.0    
google-api-core          1.3.0    
google-api-python-client 1.6.7    
google-auth              1.4.2    
google-auth-httplib2     0.0.3    
google-auth-oauthlib     0.2.0    
google-cloud-bigquery    1.1.0    
google-cloud-core        0.28.1   
google-cloud-language    1.0.2    
google-cloud-storage     1.8.0    
google-cloud-translate   1.3.1    
google-colab             0.0.1a1  
google-resumable-media   0.3.1    
googleapis-common-pr